In [1]:
"""
第四章：使用API
"""
"""
http协议从网络服务获取信息的四种方式：
GET:就是你在浏览器中输入网址浏览网站所做的事情。可以理解成“喂，网络服务器，请按照这个网址给我信息。”
POST:基本就是当你填写表单或提交信息到网络服务器的后端程序时所做的事情。如果你用API 发起一个 POST 请求，
相当于说“请把信息保存到你的数据库里”。
PUT:PUT 请求用来更新一个对象或信息。例如，API 可能会要求用 POST 请求创建新用户，但是如果你要更新老用户的邮箱
地址，就要用 PUT 请求了。
DELETE:用于删除一个对象。
"""

'\nhttp协议从网络服务获取信息的四种方式：\nGET:就是你在浏览器中输入网址浏览网站所做的事情。可以理解成“喂，网络服务器，请按照这个网址给我信息。”\nPOST:基本就是当你填写表单或提交信息到网络服务器的后端程序时所做的事情。如果你用API 发起一个 POST 请求，\n相当于说“请把信息保存到你的数据库里”。\nPUT:PUT 请求用来更新一个对象或信息。例如，API 可能会要求用 POST 请求创建新用户，但是如果你要更新老用户的邮箱\n地址，就要用 PUT 请求了。\nDELETE:用于删除一个对象。\n'

In [2]:
'''
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re
random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    html = urlopen("http://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html)
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a",href=re.compile("^(/wiki/)((?!:).)*$"))

def getHistoryIPs(pageUrl):
    # 编辑历史页面URL链接格式是：
    # http://en.wikipedia.org/w/index.php?title=Title_in_URL&action=history
    pageUrl = pageUrl.replace("/wiki/", "")
    historyUrl = "http://en.wikipedia.org/w/index.php?title="+pageUrl+"&action=history"
    print("history url is: "+historyUrl)
    html = urlopen(historyUrl)
    bsObj = BeautifulSoup(html)
    # 找出class属性是"mw-anonuserlink"的链接
    # 它们用IP地址代替用户名
    ipAddresses = bsObj.findAll("a", {"class":"mw-anonuserlink"})
    addressList = set()
    for ipAddress in ipAddresses:
        addressList.add(ipAddress.get_text())
    return addressList

links = getLinks("/wiki/Python_(programming_language)")

while(len(links) > 0):
    for link in links:
        print("-------------------")
        historyIPs = getHistoryIPs(link.attrs["href"])
        for historyIP in historyIPs:
            print(historyIP)
            
    newLink = links[random.randint(0, len(links)-1)].attrs["href"]
    links = getLinks(newLink)
'''

'\nfrom urllib.request import urlopen\nfrom bs4 import BeautifulSoup\nimport datetime\nimport random\nimport re\nrandom.seed(datetime.datetime.now())\n\ndef getLinks(articleUrl):\n    html = urlopen("http://en.wikipedia.org"+articleUrl)\n    bsObj = BeautifulSoup(html)\n    return bsObj.find("div", {"id":"bodyContent"}).findAll("a",href=re.compile("^(/wiki/)((?!:).)*$"))\n\ndef getHistoryIPs(pageUrl):\n    # 编辑历史页面URL链接格式是：\n    # http://en.wikipedia.org/w/index.php?title=Title_in_URL&action=history\n    pageUrl = pageUrl.replace("/wiki/", "")\n    historyUrl = "http://en.wikipedia.org/w/index.php?title="+pageUrl+"&action=history"\n    print("history url is: "+historyUrl)\n    html = urlopen(historyUrl)\n    bsObj = BeautifulSoup(html)\n    # 找出class属性是"mw-anonuserlink"的链接\n    # 它们用IP地址代替用户名\n    ipAddresses = bsObj.findAll("a", {"class":"mw-anonuserlink"})\n    addressList = set()\n    for ipAddress in ipAddresses:\n        addressList.add(ipAddress.get_text())\n    return addressLis

In [3]:
'''
def getCountry(ipAddress):
    try:
        response = urlopen("http://freegeoip.net/json/"+ipAddress).read().decode('utf-8')
    except HTTPError:
        return None
    responseJson = json.loads(response)
    return responseJson.get("country_code")

links = getLinks("/wiki/Python_(programming_language)")

while(len(links) > 0):
     for link in links:
        print("-------------------")
        historyIPs = getHistoryIPs(link.attrs["href"])
        for historyIP in historyIPs:
            country = getCountry(historyIP)
            if country is not None:
                print(historyIP+" is from "+country)
    newLink = links[random.randint(0, len(links)-1)].attrs["href"]
    links = getLinks(newLink)
'''

'\ndef getCountry(ipAddress):\n    try:\n        response = urlopen("http://freegeoip.net/json/"+ipAddress).read().decode(\'utf-8\')\n    except HTTPError:\n        return None\n    responseJson = json.loads(response)\n    return responseJson.get("country_code")\n\nlinks = getLinks("/wiki/Python_(programming_language)")\n\nwhile(len(links) > 0):\n     for link in links:\n        print("-------------------")\n        historyIPs = getHistoryIPs(link.attrs["href"])\n        for historyIP in historyIPs:\n            country = getCountry(historyIP)\n            if country is not None:\n                print(historyIP+" is from "+country)\n    newLink = links[random.randint(0, len(links)-1)].attrs["href"]\n    links = getLinks(newLink)\n'

In [4]:
"""
第五章：存储数据
"""
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com")
bsObj = BeautifulSoup(html)
imageLocation = bsObj.find("a", {"id": "logo"}).find("img")["src"]
urlretrieve(imageLocation, "logo.jpg")

('logo.jpg', <http.client.HTTPMessage at 0x7f1b2efcb198>)

In [7]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
downloadDirectory = "downloaded"
baseUrl = "http://pythonscraping.com"
def getAbsoluteURL(baseUrl, source):
    if source.startswith("http://www."):
        url = "http://"+source[11:]
    elif source.startswith("http://"):
        url = source
    elif source.startswith("www."):
        url = source[4:]
        url = "http://"+source
    else:
        url = baseUrl+"/"+source
    if baseUrl not in url:
        return None
    return url

def getDownloadPath(baseUrl, absoluteUrl, downloadDirectory):
    path = absoluteUrl.replace("www.", "")
    path = path.replace(baseUrl, "")
    path = downloadDirectory+path
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    return path

html = urlopen("http://www.pythonscraping.com")
bsObj = BeautifulSoup(html)
downloadList = bsObj.findAll(src=True)
for download in downloadList:
    fileUrl = getAbsoluteURL(baseUrl, download["src"])
    if fileUrl is not None:
        print(fileUrl)
urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, downloadDirectory))

http://pythonscraping.com/misc/jquery.js?v=1.4.4
http://pythonscraping.com/misc/jquery.once.js?v=1.2
http://pythonscraping.com/misc/drupal.js?q4na2g
http://pythonscraping.com/sites/all/themes/skeletontheme/js/jquery.mobilemenu.js?q4na2g
http://pythonscraping.com/sites/all/modules/google_analytics/googleanalytics.js?q4na2g
http://pythonscraping.com/sites/default/files/lrg_0.jpg
http://pythonscraping.com/https://covers.oreillystatic.com/images/0636920078067/lrg.jpg
http://pythonscraping.com/img/lrg%20(1).jpg


('downloaded/img/lrg%20(1).jpg', <http.client.HTTPMessage at 0x7f1b2e631518>)

In [1]:
'''
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://en.wikipedia.org/wiki/Comparison_of_text_editors")
bsObj = BeautifulSoup(html)
# 主对比表格是当前页面上的第一个表格
table = bsObj.findAll("table",{"class":"wikitable"})[0]
rows = table.findAll("tr")
csvFile = open("./files/editors.csv", 'wt', newline="", encoding='utf-8')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text())
            writer.writerow(csvRow)
finally:
    csvFile.close()
'''

'\nimport csv\nfrom urllib.request import urlopen\nfrom bs4 import BeautifulSoup\nhtml = urlopen("http://en.wikipedia.org/wiki/Comparison_of_text_editors")\nbsObj = BeautifulSoup(html)\n# 主对比表格是当前页面上的第一个表格\ntable = bsObj.findAll("table",{"class":"wikitable"})[0]\nrows = table.findAll("tr")\ncsvFile = open("./files/editors.csv", \'wt\', newline="", encoding=\'utf-8\')\nwriter = csv.writer(csvFile)\ntry:\n    for row in rows:\n        csvRow = []\n        for cell in row.findAll([\'td\', \'th\']):\n            csvRow.append(cell.get_text())\n            writer.writerow(csvRow)\nfinally:\n    csvFile.close()\n'

In [4]:
import pymysql
conn = pymysql.connect(host='127.0.0.1',user='root', passwd='123456', db='mysql')
cur = conn.cursor()
cur.execute("USE scraping")
cur.execute("SELECT * FROM pages WHERE id=1")
print(cur.fetchone())
cur.close()
conn.close()

(1, 'Test page title', 'This is some te\nst page content. It can be up to 10,000 characters long.', datetime.datetime(2020, 8, 27, 9, 27, 47))


In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random
import pymysql
conn = pymysql.connect(host='127.0.0.1',user='root', passwd='123456', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute("USE scraping")
random.seed(datetime.datetime.now())
def store(title, content):
    cur.execute("INSERT INTO pages (title, content) VALUES (\"%s\",\"%s\")", (title, content))
    cur.connection.commit()
    
def getLinks(articleUrl):
    html = urlopen(articleUrl)
    bsObj = BeautifulSoup(html)
    title = bsObj.find("h1", {"class": "title-article"}).get_text()
    content = "待定"
    store(title, content)
    return bsObj.findAll("a", href=re.compile("^(https://blog.csdn.net/).*/article/details/.*$"))

links = getLinks("https://blog.csdn.net/serverke/article/details/108140525")

num = 1 
try:
    while len(links) > 0:
        num += 1
        if num > 20:
            break
        newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
        print(newArticle)
        links = getLinks(newArticle)
finally:
    cur.close()
    conn.close()

https://blog.csdn.net/qq_43800290/article/details/106147475
https://blog.csdn.net/weixin_46285416/article/details/108186216
https://blog.csdn.net/weixin_46285416/article/details/108186216#comments_13130954
https://blog.csdn.net/ThinkWon/article/details/103592572
https://blog.csdn.net/u013078871/article/details/104797901
https://blog.csdn.net/u013078871/article/details/107999093
https://blog.csdn.net/blogdevteam/article/details/103478461#博客等级
https://blog.csdn.net/blogdevteam/article/details/101446074
https://blog.csdn.net/wclin88/article/details/6401403
https://blog.csdn.net/wclin88/article/details/6751254#comments_1955290
https://blog.csdn.net/Vee__/article/details/96164605
https://blog.csdn.net/zhou120189162/article/details/82826657
https://blog.csdn.net/blogdevteam/article/details/103478461#博客等级
https://blog.csdn.net/blogdevteam/article/details/103953269#comments_13130477
https://blog.csdn.net/blogdevteam/article/details/108000176
https://blog.csdn.net/weixin_47064102/article/detail

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute("USE csdn")
def insertPageIfNotExists(url):
    cur.execute("SELECT * FROM pages WHERE url = %s", (url))
    if cur.rowcount == 0:
        cur.execute("INSERT INTO pages (url) VALUES (%s)", (url))
        conn.commit()
        return cur.lastrowid
    else:
        return cur.fetchone()[0]
    
def insertLink(fromPageId, toPageId):
    cur.execute("SELECT * FROM links WHERE fromPageId = %s AND toPageId = %s",(int(fromPageId), int(toPageId)))
    if cur.rowcount == 0:
        cur.execute("INSERT INTO links (fromPageId, toPageId) VALUES (%s, %s)",(int(fromPageId), int(toPageId)))
        conn.commit()
        
pages = set()
num = 1
def getLinks(pageUrl, recursionLevel):
    global pages
    global num
    num += 1
    if num > 20:
        return
    if recursionLevel > 4:
        return;
    pageId = insertPageIfNotExists(pageUrl)
    html = urlopen(pageUrl)
    bsObj = BeautifulSoup(html)
    for link in bsObj.findAll("a",href=re.compile("^(https://blog.csdn.net/).*/article/details/.*$")):
        insertLink(pageId, insertPageIfNotExists(link.attrs['href']))
        if link.attrs['href'] not in pages:
        # 遇到一个新页面，加入集合并搜索里面的词条链接
            newPage = link.attrs['href']
            pages.add(newPage)
            getLinks(newPage, recursionLevel+1)
        
getLinks("https://blog.csdn.net/serverke/article/details/108140525", 0)
cur.close()
conn.close()

In [5]:
'''
需要本地有smtp客户端，不然发送不了邮件
import smtplib
from email.mime.text import MIMEText
msg = MIMEText("The body of the email is here")
msg['Subject'] = "An Email Alert"
msg['From'] = "ryan@pythonscraping.com"
msg['To'] = "webmaster@pythonscraping.com"
s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()
'''

'\n需要本地有smtp客户端，不然发送不了邮件\nimport smtplib\nfrom email.mime.text import MIMEText\nmsg = MIMEText("The body of the email is here")\nmsg[\'Subject\'] = "An Email Alert"\nmsg[\'From\'] = "ryan@pythonscraping.com"\nmsg[\'To\'] = "webmaster@pythonscraping.com"\ns = smtplib.SMTP(\'localhost\')\ns.send_message(msg)\ns.quit()\n'

In [6]:
'''
第六章：读取文档
'''
from urllib.request import urlopen
textPage = urlopen("http://www.pythonscraping.com/pages/warandpeace/chapter1.txt")
print(textPage.read())

b'CHAPTER I\n\n"Well, Prince, so Genoa and Lucca are now just family estates of theBuonapartes. But I warn you, if you don\'t tell me that this means war,if you still try to defend the infamies and horrors perpetrated bythat Antichrist- I really believe he is Antichrist- I will havenothing more to do with you and you are no longer my friend, no longermy \'faithful slave,\' as you call yourself! But how do you do? I seeI have frightened you- sit down and tell me all the news."\n\nIt was in July, 1805, and the speaker was the well-known AnnaPavlovna Scherer, maid of honor and favorite of the Empress MaryaFedorovna. With these words she greeted Prince Vasili Kuragin, a manof high rank and importance, who was the first to arrive at herreception. Anna Pavlovna had had a cough for some days. She was, asshe said, suffering from la grippe; grippe being then a new word inSt. Petersburg, used only by the elite.\n\nAll her invitations without exception, written in French, anddelivered by a scarle

In [7]:
"""
unicode:utf-8:UTF-8 的每个字符开头有一个标记表示“这个字符只用一个字节”或“那个字符需要用两个字节”，一个字符最多可以是四个字节。
ASCII:8位(原本7位,后来8位)，第一位为ASCII 文档里的“填充位”，让所有以“0”开头的字节表示这个字符只用 1 个字节，从而把 ASCII 和 UTF-8 编码完美地结合在一起。
"""
from urllib.request import urlopen
textPage = urlopen("http://www.pythonscraping.com/pages/warandpeace/chapter1-ru.txt")
print(str(textPage.read(), 'utf-8'))

ЧАСТЬ ПЕРВАЯ

I

— Eh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous préviens que si vous ne me dites pas que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocités de cet Antichrist (ma parole, j'y crois) — je ne vous connais plus, vous n'êtes plus mon ami, vous n'êtes plus мой верный раб, comme vous dites. Ну, здравствуйте, здравствуйте. Je vois que je vous fais peur, садитесь и рассказывайте.
Так говорила в июле 1805 года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп, как она говорила (грипп был тогда новое слово, употреблявшееся только редкими). В записочках, разосланных утром с красным лакеем, было написано без различия во всех:
«Si vous n'avez rien de mieux à faire, Monsieur le comte (или mon 

In [11]:
from urllib.request import urlopen
from io import StringIO
import csv
data = urlopen("http://pythonscraping.com/files/MontyPythonAlbums.csv").read().decode('ascii', 'ignore')
dataFile = StringIO(data)
csvReader = csv.reader(dataFile)
for row in csvReader:
    print(row)

['Name', 'Year']
["Monty Python's Flying Circus", '1970']
['Another Monty Python Record', '1971']
["Monty Python's Previous Record", '1972']
['The Monty Python Matching Tie and Handkerchief', '1973']
['Monty Python Live at Drury Lane', '1974']
['An Album of the Soundtrack of the Trailer of the Film of Monty Python and the Holy Grail', '1975']
['Monty Python Live at City Center', '1977']
['The Monty Python Instant Record Collection', '1977']
["Monty Python's Life of Brian", '1979']
["Monty Python's Cotractual Obligation Album", '1980']
["Monty Python's The Meaning of Life", '1983']
['The Final Rip Off', '1987']
['Monty Python Sings', '1989']
['The Ultimate Monty Python Rip Off', '1994']
['Monty Python Sings Again', '2014']


In [9]:
from urllib.request import urlopen
from io import StringIO
import csv
import pandas as pd
data = urlopen("http://pythonscraping.com/files/MontyPythonAlbums.csv").read().decode('ascii', 'ignore')
dataFile = StringIO(data)
csvReader = pd.read_csv(dataFile)
csvReader

,Name,Year
0,Monty Python's Flying Circus,1970
1,Another Monty Python Record,1971
2,Monty Python's Previous Record,1972
3,The Monty Python Matching Tie and Handkerchief,1973
4,Monty Python Live at Drury Lane,1974
5,An Album of the Soundtrack of the Trailer of t...,1975
6,Monty Python Live at City Center,1977
7,The Monty Python Instant Record Collection,1977
8,Monty Python's Life of Brian,1979
9,Monty Python's Cotractual Obligation Album,1980


In [1]:
from urllib.request import urlopen
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open

def readPDF(pdfFile):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    process_pdf(rsrcmgr, device, pdfFile)
    device.close()
    content = retstr.getvalue()
    retstr.close()
    return content

pdfFile = urlopen("http://pythonscraping.com/pages/warandpeace/chapter1.pdf")
outputString = readPDF(pdfFile)
print(outputString)
pdfFile.close()

CHAPTER I

"Well, Prince, so Genoa and Lucca are now just family estates of 
theBuonapartes. But I warn you, if you don't tell me that this 
means war,if you still try to defend the infamies and horrors 
perpetrated bythat Antichrist- I really believe he is Antichrist- I will 
havenothing more to do with you and you are no longer my friend, 
no longermy 'faithful slave,' as you call yourself! But how do you 
do? I seeI have frightened you- sit down and tell me all the news."

It was in July, 1805, and the speaker was the well-known 
AnnaPavlovna Scherer, maid of honor and favorite of the 
Empress MaryaFedorovna. With these words she greeted Prince 
Vasili Kuragin, a manof high rank and importance, who was the 
first to arrive at herreception. Anna Pavlovna had had a cough for 
some days. She was, asshe said, suffering from la grippe; grippe 
being then a new word inSt. Petersburg, used only by the elite.

All her invitations without exception, written in French, 
anddelivered by a scar

In [4]:
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO
from bs4 import BeautifulSoup
wordFile = urlopen("http://pythonscraping.com/pages/AWordDocument.docx").read()
wordFile = BytesIO(wordFile)
document = ZipFile(wordFile)
xml_content = document.read('word/document.xml')
#print(xml_content.decode('utf-8'))
wordObj = BeautifulSoup(xml_content.decode('utf-8'))
textStrings = wordObj.findAll("w:t")
for textElem in textStrings:
    print(textElem.text)

A Word Document on a Website
This is a Word document, full of content that you want very much. Unfortunately, it’s difficult to access because I’m putting it on my website as a .
docx
 file, rather than just publishing it as HTML
